In [ ]:
#| default_exp extend

# extend

> Extend an already existing `pd.DataFrame` with new data

In [ ]:
#| export
import pathlib

import pandas as pd
import yaml

import sproc.structure
import sproc.bundle
import sproc.hier
import sproc.io
import sproc.assemble
import sproc.postprocess

These `import`s are not actually needed by the final *exported* code

In [ ]:
import itertools

## Historical data

Directory from which to read historical data

In [ ]:
history_directory = pathlib.Path.cwd().parent / 'samples'
# history_directory = pathlib.Path.cwd() / 'reports'
assert history_directory.exists()
print(history_directory)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples


Input directory for the `pd.DataFrame` to be updated

In [ ]:
history_input_file = history_directory / '2018-2021_20samples.parquet'
# history_input_file = history_directory / '2018-2021.parquet'
assert history_input_file.exists()
print(history_input_file)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/2018-2021_20samples.parquet


The specific file is

In [ ]:
history_df = pd.read_parquet(history_input_file)
history_df.head(2)

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                            452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                           updated  \
                                                 

In [ ]:
# history_df = pd.concat({'some.zip': history_df}, names=['zip'])
# history_df.to_parquet(history_input_file)

## New data

The directory for the zip file whose data is to appended to the already existing `pd.DataFrame`

In [ ]:
zip_directory = history_directory
# zip_directory = pathlib.Path.cwd() / 'data' / 'agregados'
assert zip_directory.exists()
print(zip_directory)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples


The particular file

In [ ]:
zip_file = zip_directory / 'PlataformasAgregadasSinMenores_202201_05-06.zip'
assert zip_file.exists()
print(zip_file)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PlataformasAgregadasSinMenores_202201_05-06.zip


The file is read and processed in the usual way to get a `pd.DataFrame`

In [ ]:
new_df = sproc.bundle.read_zip(zip_file, concatenate=True)

*Multindexed* columns are built

In [ ]:
new_df = sproc.hier.flat_df_to_multiindexed_df(new_df)

Number of levels in the columns

In [ ]:
new_df.columns.nlevels

6

Information about deleted entries

In [ ]:
deleted_series = sproc.bundle.read_deleted_zip(zip_file)

## Merge

Old and new data are stacked together

In [ ]:
concatenated_df = sproc.assemble.stack(history_df, new_df)
concatenated_df

id  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
zip                                             file name                                          entry                                                      
some.zip                                        PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   452    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   451    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   450    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   449    https://contrataciondelestado.es/sindicacion/P...   
..                                                                                                                                                      ...   
PlataformasAgregadasSinMenores_202201_05-06.zip PlataformasAgregadasSinMenores_20220106_030013.... 471    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   472    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   473    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   474    https://contrataciondelestado.es/sindicacion/P...   
                                                                                                   475    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                                                                    summary  \
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                                                                                                                                                              
                  

Only the last update is kept

In [ ]:
concatenated_df = sproc.postprocess.keep_updates_only(concatenated_df)
concatenated_df.head(2)

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                            452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                           updated  \
                                                 

In order to make it easy to update the `deleted_on` column in the existing `pd.DataFrame` using the data contained in the *deleted* `pd.Series` (i.e., to get *automatic 
alignment*) we
- reindex the `DataFrame` using `id`

In [ ]:
reindexed_concatenated_df = concatenated_df.reset_index().set_index(['id'])
reindexed_concatenated_df.head(2)

zip  \
                                                               
                                                               
                                                               
                                                               
                                                               
                                                               
                                                               
id                                                             
https://contrataciondelestado.es/sindicacion/Pl...  some.zip   
https://contrataciondelestado.es/sindicacion/Pl...  some.zip   

                                                                                            file name  \
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
id                                                                                                      
https://contrataciondelestado.es/sindicacion/Pl...  PlataformasAgregadasSinMenores_20180217_180137...   
https://contrataciondelestado.es/sindicacion/Pl...  PlataformasAgregadasSinMenores_20180217_180137...   

                                                   entry  \
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
                                                           
id                                                         
https://contrataciondelestado.es/sindicacion/Pl...   453   
https://contrataciondelestado.es/sindicacion/Pl...   452   

                                                                                              summary  \
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
                                                                                                        
id                                                                                                      
https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1284/17, Entidad: Diputación Provi...   
https://contrataciondelestado.es/sindicacion/Pl...  Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                title  \
                                                                                                        
                                                                                                        
                                                                

- drop the first level of the series (which leaves only `id`)

In [ ]:
# reindexed_deleted_series = deleted_series.droplevel(0)
reindexed_deleted_series = deleted_series.droplevel([0, 1])
reindexed_deleted_series.head(2)

id
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6724977   2022-01-04 00:12:01.376000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968315   2022-01-03 23:11:57.567000+00:00
Name: deleted_on, dtype: datetime64[ns, UTC]

Duplicates are removed from the above `pd.Series`

In [ ]:
deduplicated_reindexed_deleted_series = sproc.postprocess.deduplicate_deleted_series(reindexed_deleted_series)
deduplicated_reindexed_deleted_series

id
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1957921   2022-01-03 23:11:56.535000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1959074   2022-01-03 23:11:56.497000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968266   2022-01-04 23:12:11.916000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968267   2022-01-04 23:12:12.775000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968268   2022-01-04 23:12:12.731000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968269   2022-01-04 23:12:13.013000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968271   2022-01-04 23:12:12.172000+00:00
https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968272   2022-01-04 23:12:12.957000+00:00
https://contrataciondelestado.es/sindicacion/

The old and new sizes

In [ ]:
reindexed_deleted_series.shape, deduplicated_reindexed_deleted_series.shape

((49,), (49,))

No keys are lost along the way

In [ ]:
assert (set(reindexed_deleted_series.index) - set(deduplicated_reindexed_deleted_series.index)) == set()

Duplicates in the old `pd.Series`

In [ ]:
reindexed_deleted_series.loc[reindexed_deleted_series.index.duplicated()]

Series([], Name: deleted_on, dtype: datetime64[ns, UTC])

There should be none in the new

In [ ]:
assert deduplicated_reindexed_deleted_series.loc[deduplicated_reindexed_deleted_series.index.duplicated()].empty

What is the intersection between the entries in the new `pd.DataFrame` and the `pd.Series` with information about deletes (only the first elements).

In [ ]:
list(itertools.islice(set(reindexed_concatenated_df.index) & set(deduplicated_reindexed_deleted_series.index), 0, 4))

[]

A deleted entry, identified by its `id`, and also showing the source *ATOM* file

In [ ]:
deleted_entry = deduplicated_reindexed_deleted_series.index[0]
deleted_entry

'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1957921'

It's not actually present in the data

In [ ]:
(new_df['id'] == deleted_entry).sum()

0

In order to see if something changes when updating column `deleted_on`, a backup of the old `pd.DataFrame` is made

In [ ]:
# reindexed_concatenated_df = bak_reindexed_concatenated_df.copy()
bak_reindexed_concatenated_df = reindexed_concatenated_df.copy()

The columns `deleted_on` in the `pd.DataFrame` is updated for the indexes present in the *deleted series*. Notice that we cannot do

```
reindexed_concatenated_df['deleted_on'] = deduplicated_reindexed_deleted_series
```

because that would effectively wipe out the values of all the entries not present in the series.

*na*'s in the existing `pd.DataFrame` are filled in with data from the *deleted series*

In [ ]:
reindexed_concatenated_df['deleted_on'] = reindexed_concatenated_df['deleted_on'].fillna(deduplicated_reindexed_deleted_series)

The number of *non-NAs* in the old and new `pd.DataFrame`s

In [ ]:
n_notnas_old, n_notnas_new = bak_reindexed_concatenated_df['deleted_on'].notna().sum(), reindexed_concatenated_df['deleted_on'].notna().sum()
n_notnas_old, n_notnas_new

(0, 0)

The latter be larger than or equal than the former

In [ ]:
assert n_notnas_new >= n_notnas_old

The original index is reset (notice the `reset_index()` in the middle that avoids losing `id` columns, which is right now the index)

In [ ]:
reindexed_concatenated_df = reindexed_concatenated_df.reset_index().set_index(['zip', 'file name', 'entry'])

Finally

In [ ]:
reindexed_concatenated_df.head(2)

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                            452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                           updated  \
                                                 

A function encompassing all of the above steps in one go

In [ ]:
#| export
def df_with_zip(history_df: pd.DataFrame, zip_file: str | pathlib.Path, output_file: str | pathlib.Path | None = None) -> None | pd.DataFrame:

    # in case `str`s were passed
    zip_file = pathlib.Path(zip_file)

    assert zip_file.exists(), f"can't find {zip_file}"
    
    # new data is parsed into a `pd.DataFrame`...
    new_df = sproc.bundle.read_zip(zip_file, concatenate=True)
    
    # ...whose columns are a *multiindex*
    new_df = sproc.hier.flat_df_to_multiindexed_df(new_df)
    
    # we also need the information about which entries are flagged as deleted
    deleted_series = sproc.bundle.read_deleted_zip(zip_file)
    
    # the time zone for the above deleted series and the column "deleted_on" in the historical data
    history_deleted_timezone = history_df["deleted_on"].dt.tz
    deleted_series_timezone = deleted_series.dt.tz
    
    # if there is any actual value in the "deleted on" column...
    if history_deleted_timezone:
    
        # assert deleted_series.dt.tz == history_deleted_timezone, f'{deleted_series.dt.tz=}, {history_deleted_timezone=}'
        
        if history_deleted_timezone != deleted_series_timezone:

            print(f'converting time zone in zip file {deleted_series_timezone} to {history_deleted_timezone} (that in the historical file)...')
            deleted_series = deleted_series.dt.tz_convert(history_deleted_timezone)
    
    # the two DataFrames are stacked together
    concatenated_df = sproc.assemble.stack(history_df, new_df)
    
    # the same contract might show up more than once due to updates...but only the last one is kept
    concatenated_df = sproc.postprocess.keep_updates_only(concatenated_df)
    
    # duplicates are dropped from the deleted series
    deduplicated_deleted_series = sproc.postprocess.deduplicate_deleted_series(deleted_series)
    
    # for the sake of flagging deleted entries, the new (concatenated) DataFrame and the "deleted series" are indexed the same
    reindexed_concatenated_df = concatenated_df.reset_index().set_index(['id'])
    # deduplicated_reindexed_deleted_series = deduplicated_deleted_series.droplevel(level='file name')
    deduplicated_reindexed_deleted_series = deduplicated_deleted_series.droplevel(level=['zip', 'file name'])
    
    # the `deleted_on` column in the DataFrame is filled in whenever appropriate
    reindexed_concatenated_df['deleted_on'] = reindexed_concatenated_df['deleted_on'].fillna(deduplicated_reindexed_deleted_series)
    
    # the original index is set back in place
    # res = reindexed_concatenated_df.reset_index().set_index(['file name', 'entry'])
    res = reindexed_concatenated_df.reset_index().set_index(['zip', 'file name', 'entry'])
    
    if output_file:
        
        # so that we can exploit pathlib's API...
        output_file = pathlib.Path(output_file)
        
        # ...for checking stuff
        assert output_file.suffix == '.parquet', f'output file should end in ".parquet"'
    
        parquet_df = sproc.assemble.parquet_amenable(res)
        
        parquet_df.to_parquet(output_file)
        
    else:
    
        return res

For historical reasons, a slightly higher-level function is included

In [ ]:
#| export
def parquet_with_zip(history_file: str | pathlib.Path, zip_file: str | pathlib.Path, output_file: str | pathlib.Path | None = None) -> None | pd.DataFrame:
    
    # in case `str`s were passed
    history_file = pathlib.Path(history_file)
    
    assert history_file.exists(), f"can't find {history_file}"
    
    # historical data
    history_df = pd.read_parquet(history_file)

    return df_with_zip(history_df, zip_file, output_file)    

In [ ]:
merged_df = parquet_with_zip(history_input_file, zip_file)
merged_df.head(2)

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                            452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                           updated  \
                                                 

We should be getting exactly the same result as above

In [ ]:
assert reindexed_concatenated_df.equals(merged_df)

In [ ]:
output_file = history_directory / 'merged.parquet'
parquet_with_zip(history_input_file, zip_file, output_file)

In [ ]:
sample_output_df = pd.read_parquet(output_file)
sample_output_df.head(2)

id  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    https://contrataciondelestado.es/sindicacion/P...   
                                                            452    https://contrataciondelestado.es/sindicacion/P...   

                                                                                                             summary  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Expediente: 1284/17, Entidad: Diputación Provi...   
                                                            452    Expediente: 1282/17, Entidad: Diputación Provi...   

                                                                                                               title  \
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
                                                                                                                       
zip      file name                                          entry                                                      
some.zip PlataformasAgregadasSinMenores_20180217_180137_... 453    Refuerzo de Firme en la VP 3001 Renedo de Esgu...   
                                                            452    Refuerzo de Firme en la VP 6603 Mota del Marqu...   

                                                                                           updated  \
                                                 

Let us make sure nothing was lost

In [ ]:
assert sample_output_df.shape == merged_df.shape

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('70_extend.ipynb')